In [36]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=512,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [37]:
from langchain_community.document_loaders import WikipediaLoader

docs = WikipediaLoader(query="Leonardo Di Caprio").load_and_split(text_splitter = text_splitter )

In [ ]:
# List to hold the content of each document
doc_contents = [doc.page_content for doc in docs]

# List to hold the IDs for each document
doc_ids = list(range(1, len(docs) + 1))

In [ ]:
doc_contents[0]

"Leonardo Wilhelm DiCaprio (; Italian: [diˈkaːprjo]; born November 11, 1974) is an American actor and film producer. Known for his work in biographical and period films, he is the recipient of numerous accolades, including an Academy Award, a British Academy Film Award, and three Golden Globe Awards. As of 2019, his films have grossed over $7.2 billion worldwide, and he has been placed eight times in annual rankings of the world's highest-paid actors."

In [ ]:
from qdrant_client import QdrantClient

# Initialize the client
client = QdrantClient(":memory:")

client.add(
    collection_name="leo_collection",
    documents=doc_contents,
    ids=doc_ids,
)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [ ]:
from dspy.retrieve.qdrant_rm import QdrantRM
import dspy

qdrant_retriever_model = QdrantRM("leo_collection", client, k=10)

ollama_model = dspy.OllamaLocal(model="llama3",model_type='text',
                                max_tokens=350,
                                temperature=0.1,
                                top_p=0.8, frequency_penalty=1.17, top_k=40)

dspy.settings.configure(lm= ollama_model, rm=qdrant_retriever_model)

In [ ]:
import dspy
def get_top_passages(question):
    retrieve = dspy.Retrieve(k=10)
    topK_passages = retrieve(question,k=10).passages
    print(f"Top {retrieve.k} passages for question: {question} \n", '-' * 30, '\n')
    for idx, passage in enumerate(topK_passages):
        print(f'{idx+1}]', passage, '\n')

In [ ]:
get_top_passages("Where was Leo born?")

Top 10 passages for question: Where was Leo born? 
 ------------------------------ 

1] == Early life == 

2] == Early life and acting background == 

3] == Biography ==
DiCaprio was born to George Leon DiCaprio and Olga Anne Jacobs. His father was the son of Italian immigrants, Salvatore Di Caprio and Rosina Cassella, and his mother was of German descent. 

4] Leonardo Wilhelm DiCaprio was born on November 11, 1974, in Los Angeles, California. He is the only child of Irmelin Indenbirken, a legal secretary, and George DiCaprio, an underground comix artist and distributor; they met while attending college and moved to Los Angeles after graduating. His mother is German and his father is of Italian and German descent. His maternal grandfather, Wilhelm Indenbirken, was German, and his maternal grandmother, Helene Indenbirken, was a Russian immigrant living in 

5] DiCaprio's parents named him Leonardo because his pregnant mother first felt him kick while she was looking at a Leonardo da Vi

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField()

In [ ]:
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

In [ ]:
from dsp.utils import deduplicate

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops

    def forward(self, question):
        context = []

        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

In [ ]:
# Ask any question you like to this simple RAG program.
my_question = "Give me all the co-actors of Leonardo Di Caprio in the movie in which one of his co-stars was Robert De Niro?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
uncompiled_baleen = SimplifiedBaleen()  # uncompiled (i.e., zero-shot) program
pred = uncompiled_baleen(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x33b314550>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
ollama_model.inspect_history(n=3)

''